In [1]:
"""
03 - Agent Test
This notebook tests the RAG agent logic, processing natural language queries
against the enriched transaction data and vector store.
"""

import sys
from pathlib import Path

# Add src/ to Python path
sys.path.append(str(Path().resolve().parent))

from src.agent import process_query
from src.vector_store import load_existing_vector_store

print("✅ Successfully imported agent and vector_store modules.")

✅ Successfully imported agent and vector_store modules.


In [2]:
from pathlib import Path

# Define paths to vector store + metadata
VECTOR_STORE_FILE = Path("../data/processed/vector_store.faiss")
METADATA_FILE = Path("../data/processed/metadata.pkl")

# Load FAISS index and metadata DataFrame
index, metadata = load_existing_vector_store(
    vector_store_file=VECTOR_STORE_FILE,
    metadata_file=METADATA_FILE
)

print(f"✅ FAISS index loaded with {index.ntotal} vectors.")
print("Metadata preview:")
display(metadata.head())

✅ Loaded FAISS index and metadata successfully.
✅ FAISS index loaded with 102 vectors.
Metadata preview:


,text,Dollars,Date,Account,Type,Category,Vendor,Tags,Amount,weekday,is_weekend,is_fixed
0,"On 2025-04-01, you spent $775.00 at 6640 pasad...",($775.00),2025-04-01,usaa s,expense,rent,6640 pasado,NaN,775.00,Tuesday,False,True
1,"On 2025-04-01, you spent $11.00 at coral tree ...",($11.00),2025-04-01,usaa c,expense,food,coral tree,eating out,11.00,Tuesday,False,False
2,"On 2025-04-02, you spent $5.99 at spotify (Cat...",($5.99),2025-04-02,usaa c,expense,subscription,spotify,NaN,5.99,Wednesday,False,True
3,"On 2025-04-03, you spent $5.30 at ucsb (Catego...",($5.30),2025-04-03,usaa c,expense,parking,ucsb,NaN,5.30,Thursday,False,False
4,"On 2025-04-07, a transaction of $83.99 occurre...",$83.99,2025-04-07,usaa c,reimburstment,lodging,slo hotel,NaN,-83.99,Monday,False,False


In [3]:
# Quick check: are index and metadata loaded?
print("index:", '✅ Defined' if 'index' in globals() else '❌ Not defined')
print("metadata:", '✅ Defined' if 'metadata' in globals() else '❌ Not defined')

index: ✅ Defined
metadata: ✅ Defined


In [4]:
# Test a query expected to match an intent
query = "How much did I spend on eating out last month?"

response = process_query(
    query=query,
    metadata=metadata,
    vector_index=index
)

# Display result
if response["type"] == "formula":
    print(f"🔢 Answer: {response['result']}")
    print(f"🧮 Formula: {response['formula']}")
    print(f"💬 Follow-up: {response['follow_up']}")
else:
    print(response["follow_up"])
    display(response["result"])

✅ Matched intent: \bhow (?:much|many).*?(?:eat|eating out|fast food)\b
🔢 Answer: 206.31
🧮 Formula: Σ Amount where Tags~='eating out'
💬 Follow-up: I used `Σ Amount where Tags~='eating out'`. Need any tweaks—different tags, date range, etc.?


In [5]:
# Test a query likely to fall back to semantic search
query = "Show me what I spent at Blenders."

response = process_query(
    query=query,
    metadata=metadata,
    vector_index=index
)

# Display result
if response["type"] == "formula":
    print(f"🔢 Answer: {response['result']}")
    print(f"🧮 Formula: {response['formula']}")
    print(f"💬 Follow-up: {response['follow_up']}")
else:
    print(response["follow_up"])
    display(response["result"])

ℹ️ No intent match; falling back to vector search...
I found these transactions that seem relevant. Would you like to filter or refine?


,text,Dollars,Date,Account,Type,Category,Vendor,Tags,Amount,weekday,is_weekend,is_fixed,similarity_score
0,"On 2025-04-08, you spent $9.00 at blenders (Ca...",($9.00),2025-04-08,usaa c,expense,food,blenders,eating out,9.00,Tuesday,False,False,0.445830
1,"On 2025-04-14, you spent $123.84 at sephora (C...",($123.84),2025-04-14,usaa c,expense,wellness,sephora,NaN,123.84,Monday,False,False,0.359342
2,"On 2025-04-07, you spent $36.64 at costco (Cat...",($36.64),2025-04-07,usaa c,expense,fuel,costco,NaN,36.64,Monday,False,False,0.346576
3,"On 2025-04-21, you spent $47.50 at richies bar...",($47.50),2025-04-21,usaa c,expense,wellness,richies barbershop,NaN,47.50,Monday,False,False,0.342922
4,"On 2025-04-14, you spent $8.37 at convenience ...",($8.37),2025-04-14,usaa c,expense,food,convenience store,"yosemite, beverage",8.37,Monday,False,False,0.342548


In [6]:
# Loop over several queries for fast testing
test_queries = [
    "How much did I spend on fast food?",
    "What portion of my income went to food?",
    "What were my biggest expenses at Costco?",
    "List transactions related to subscriptions.",
    "Did I spend anything on rent?",
]

for query in test_queries:
    print(f"\n🔎 Query: {query}")
    response = process_query(
        query=query,
        metadata=metadata,
        vector_index=index
    )
    if response["type"] == "formula":
        print(f"🔢 Answer: {response['result']}")
        print(f"🧮 Formula: {response['formula']}")
        print(f"💬 Follow-up: {response['follow_up']}")
    else:
        print(response["follow_up"])
        display(response["result"])



🔎 Query: How much did I spend on fast food?
✅ Matched intent: \bhow (?:much|many).*?(?:eat|eating out|fast food)\b
🔢 Answer: 206.31
🧮 Formula: Σ Amount where Tags~='eating out'
💬 Follow-up: I used `Σ Amount where Tags~='eating out'`. Need any tweaks—different tags, date range, etc.?

🔎 Query: What portion of my income went to food?
✅ Matched intent: \bportion\b.*\bincome\b.*\bfood\b
🔢 Answer: nan
🧮 Formula: Σ Amount(food) / |Σ Amount(income)|
💬 Follow-up: I used `Σ Amount(food) / |Σ Amount(income)|`. Need any tweaks—different tags, date range, etc.?

🔎 Query: What were my biggest expenses at Costco?
ℹ️ No intent match; falling back to vector search...
I found these transactions that seem relevant. Would you like to filter or refine?


,text,Dollars,Date,Account,Type,Category,Vendor,Tags,Amount,weekday,is_weekend,is_fixed,similarity_score
0,"On 2025-04-23, you spent $49.03 at costco (Cat...",($49.03),2025-04-23,usaa c,expense,food,costco,groceries,49.03,Wednesday,False,False,0.653288
1,"On 2025-04-09, you spent $7.59 at costco (Cate...",($7.59),2025-04-09,usaa c,expense,food,costco,"eating out, fast food",7.59,Wednesday,False,False,0.651466
2,"On 2025-04-07, you spent $36.64 at costco (Cat...",($36.64),2025-04-07,usaa c,expense,fuel,costco,NaN,36.64,Monday,False,False,0.599751
3,"On 2025-04-09, you spent $99.83 at costco (Cat...",($99.83),2025-04-09,usaa c,expense,food,costco,yosemite,99.83,Wednesday,False,False,0.588248
4,"On 2025-04-10, you spent $30.02 at costco (Cat...",($30.02),2025-04-10,usaa c,expense,transportation,costco,yosemite,30.02,Thursday,False,False,0.576231



🔎 Query: List transactions related to subscriptions.
ℹ️ No intent match; falling back to vector search...
I found these transactions that seem relevant. Would you like to filter or refine?


,text,Dollars,Date,Account,Type,Category,Vendor,Tags,Amount,weekday,is_weekend,is_fixed,similarity_score
0,"On 2025-04-23, a transaction of $16.00 occurre...",$16.00,2025-04-23,usaa c,reimburstment,other,ben a,yosemite,-16.00,Wednesday,False,False,0.283855
1,"On 2025-04-11, a transaction of $43.42 occurre...",$43.42,2025-04-11,usaa c,reimburstment,other,venmo,NaN,-43.42,Friday,False,False,0.244356
2,"On 2025-04-14, a transaction of $115.00 occurr...",$115.00,2025-04-14,usaa c,reimburstment,food,hot pot,eating out,-115.00,Monday,False,False,0.239996
3,"On 2025-04-07, a transaction of $83.99 occurre...",$83.99,2025-04-07,usaa c,reimburstment,lodging,slo hotel,NaN,-83.99,Monday,False,False,0.234488
4,"On 2025-04-02, you spent $5.99 at spotify (Cat...",($5.99),2025-04-02,usaa c,expense,subscription,spotify,NaN,5.99,Wednesday,False,True,0.199739



🔎 Query: Did I spend anything on rent?
ℹ️ No intent match; falling back to vector search...
I found these transactions that seem relevant. Would you like to filter or refine?


,text,Dollars,Date,Account,Type,Category,Vendor,Tags,Amount,weekday,is_weekend,is_fixed,similarity_score
0,"On 2025-04-01, you spent $775.00 at 6640 pasad...",($775.00),2025-04-01,usaa s,expense,rent,6640 pasado,NaN,775.00,Tuesday,False,True,0.437686
1,"On 2025-04-25, you spent $3.00 at state street...",($3.00),2025-04-25,usaa c,expense,parking,state street,NaN,3.00,Friday,False,False,0.297088
2,"On 2025-04-07, you spent $54.62 at state stree...",($54.62),2025-04-07,usaa c,expense,clothing,state street,second hand,54.62,Monday,False,False,0.288405
3,"On 2025-04-09, you spent $22.00 at wifi (Categ...",($22.00),2025-04-09,usaa c,expense,utilities,wifi,NaN,22.00,Wednesday,False,False,0.278654
4,"On 2025-04-10, you spent $30.02 at costco (Cat...",($30.02),2025-04-10,usaa c,expense,transportation,costco,yosemite,30.02,Thursday,False,False,0.272795


In [7]:
from src.dsl import amt

# Test amt(cat='food')
print(amt(cat='food')(metadata))

# Test amt(type_='income')
print(amt(type_='income')(metadata))

1      11.00
7      17.19
11      7.80
12      9.00
30      7.59
32     22.88
33     99.83
34      8.70
35      8.70
36     12.56
45   -115.00
48      8.37
49     17.23
52    115.61
57      8.70
59     16.31
60      2.53
65     13.05
69      2.53
72     18.71
73     27.48
75      9.50
80     21.01
81     11.63
83      4.03
84      6.39
85      6.53
86     11.63
88     10.88
89     27.06
91     49.03
98     23.57
Name: Amount, dtype: float64
13    -240.26
18     -60.06
82   -1004.00
Name: Amount, dtype: float64
